Let's tinker with collections a bit. 
1. build a collection
2. add docs (Let's try to add these without ids)
3. try to retrieve docs


In [1]:
from chromadb import PersistentClient, EmbeddingFunction, Embeddings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from typing import List

MODEL_NAME = "Alibaba-NLP/gte-Qwen2-1.5B-instruct"
DB_PATH = "./.chroma_db"

class Protocol:
    def __init__(self, path: str, text:str):
        self.path = path
        self.text  = text

class CustomEmbeddingClass(EmbeddingFunction):
    def __init__(self, model_name):
        self.embedding_model = HuggingFaceEmbedding(model_name)

    def __call__(self, input_texts: List[str])->Embeddings:
        return [self.embedding_model.get_text_embedding(text) for text in input_texts]
    




db = PersistentClient(path=DB_PATH)

custom_embedding_function = CustomEmbeddingClass(MODEL_NAME)

#collection = db.get_or_create_collection(name='SOPs', embedding_function=custom_embedding_function)

C:\Users\grace\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

oading checkpoint shards:   0%|                                                                 | 0/2 [00:00<?, ?it/s]

OSError: Insufficient system resources exist to complete the requested service. (os error 1450)

In [ ]:
import os
import json
from pathlib import Path

def documents_to_json(folder_path):
    """
    Reads all text files in a folder (including subfolders) and stores their content
    in a JSON object along with their relative paths.
    
    Args:
        folder_path (str): Path to the folder containing documents
        
    Returns:
        dict: JSON-compatible dictionary with file paths and contents
    """
    # Convert the folder path to a Path object
    base_path = Path(folder_path)
    
    # Dictionary to store the results
    documents = {}
    
    # Supported text file extensions
    text_extensions = {'.txt', '.md', }
    
    try:
        # Walk through all files in the folder and subfolders
        for file_path in base_path.rglob('*'):
            # Check if it's a file and has a text extension
            if file_path.is_file() and file_path.suffix.lower() in text_extensions:
                try:
                    # Get the relative path from the base folder
                    relative_path = str(file_path.relative_to(base_path))
                    
                    # Read the file content
                    with open(file_path, 'r', encoding='utf-8') as file:
                        content = file.read()
                    
                    # Store in dictionary
                    documents[relative_path] = content
                    
                except Exception as e:
                    print(f"Error reading file {file_path}: {str(e)}")
                    continue
        
        # Convert to JSON string
        json_output = json.dumps(documents, indent=2)
        
        # Save to a JSON file
        output_path = base_path / 'documents.json'
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(json_output)
            
        return documents
        
    except Exception as e:
        print(f"Error processing folder: {str(e)}")
        return None

# Example usage
if __name__ == "__main__":
    # Replace with your folder path
    folder_path = "./documents"
    result = documents_to_json(folder_path)
    
    if result:
        print(f"Successfully processed {len(result)} documents")
        print(f"Output saved to {folder_path}/documents.json")
    else:
        print("Failed to process documents")